In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

### Load data
#### Create dataframes 
- users, songs
- frequency for users listerning to songs  
- download 
- search queries (uid, data, time - activitiy of the users)

In [6]:
user_df = pd.read_csv('../data/parsed_data/3_1uid.csv', dtype={'uid':str})
# user_song_freq = pd.read_csv('../data/parsed_data/user_song_freq.csv',dtype={'uid':str})
# song_df = pd.read_csv('../data/parsed_data/song_table.csv', sep = ',', quotechar='"', engine='python', encoding='utf-8')

In [22]:
user_df['uid'] = user_df['uid'].dropna()
user_df = user_df.loc[(user_df['uid'] != '<script>\n') 
                      & (~user_df['uid'].isnull())
                      & (user_df['uid'] > '0')]
print(min(user_df['uid']))
print(max(user_df['uid']))

100047599
99983627


In [16]:
user_df.head()

,Unnamed: 0,uid
1,1,154563989
2,2,154806874
3,3,154777984
4,4,154801899
5,5,154522980


In [4]:
song_df.head()

,song_id,song_type,song_name,singer,song_length
0,602239.0,NaN,薛凯琪,0,NaN
1,160911.0,NaN,蔡依林&周杰伦,0,NaN
2,1033156.0,NaN,汪苏泷,0,NaN
3,294622.0,NaN,DJ舞曲,0,NaN
4,517174.0,NaN,梦鸽,0,NaN


In [6]:
user_song_freq.head()

,uid,device,song_id,freq
0,751824,ar,6483029.0,385
1,168156920,ip,6792060.0,5
2,497685,ar,7207401.0,26
3,1062806,ar,6841262.0,50
4,168195436,ar,12808784.0,22


In [ ]:
user_df.head()
user_song_freq.head()

In [ ]:
# user_df.uid = user_df.uid.apply(float).apply(str)
# song_df.song_id = song_df.song_id.apply(str)
# user_song_freq.uid = user_song_freq.uid.apply(str)
# user_song_freq.song_id = user_song_freq.song_id.apply(str)

### Cleaning data
#### User table
- Identify uid with extreme song playing frequencies and flag those in user_df 

#### Song table 
- Combine songs with same song_id, and mode of the song_type (>0), name, singer, song_length (<7200) 

unique user id: 264713
unique song id: 1559989 

In [30]:
# print (user_df.shape)
# valid_user = user_df.dropna()
# valid_user = valid_user.loc[valid_user['uid'] > '0']

valid_song = song_df.loc[song_df['song_id'] > 0]
valid_song = valid_song.dropna(subset=['song_id'])
num_song = len(valid_song['song_id'].unique())
print (num_song)

1559986


In [34]:
print(max(valid_song['song_id']))
print(min(valid_song['song_id']))
print (sorted(valid_song['song_id'].unique())[:10])

2147483647.0
2.0
[2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0]


In [4]:
print (valid_user.shape)
print (valid_song.shape)

(264713, 2)
(4182826, 5)


In [ ]:
# save valid dataset 
valid_song.to_csv('../data/valid_song_dup.csv')
valid_user.to_csv('../data/valid_user_highfreq.csv')

In [ ]:
# validate user_play

In [77]:
unique_song = pd.DataFrame(columns=['song_id', 'song_type', 'song_name', 'song_length'])
unique_song.song_id = valid_song['song_id'].unique()
unique_song.song_type = 0
unique_song.song_length = 0
unique_song.head()

for i in list(unique_song['song_id']):
    temp_df = valid_song.loc[valid_song['song_id'] == i]
    song_type = temp_df['song_type'].replace(0.0, None).dropna().mode()
    
    temp_length = valid_song.dropna(subset = ['song_length'])
    temp_df = temp_length.loc[(temp_length['song_length']>0) & (temp_length['song_length']<720)]
    ave_length = int(temp_length['song_length'].mean())
    
    song_name = temp_df['song_name'].dropna().mode()
    singer = temp_df['singer'].dropna().mode()
    unique_song.loc[unique_song['song_id'] == i, 'song_type'] = song_type
    unique_song.loc[unique_song['song_id'] == i, 'song_length'] = ave_length
    unique_song.loc[unique_song['song_id'] == i, 'song_name'] = song_name
    unique_song.loc[unique_song['song_id'] == i, 'singer'] = singer


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs

KeyboardInterrupt: 

song id with extreme frequencies

In [12]:
freq_cutoff = user_song_freq['freq'].quantile(0.9999)
extreme_freq = user_song_freq.loc[(user_song_freq['freq']>=freq_cutoff)]

In [13]:
print(freq_cutoff)

1009.0


In [16]:
outlier_user = extreme_freq.uid.unique()

In [19]:
outlier_user[:10]

array(['1791497', '751824', '497685', '37025504', '1685126', '168417936',
       '1749320', '1062806', '168852806', '168890189'], dtype=object)

In [21]:
user_song_freq.loc[user_song_freq['uid']=='1791497'].head()
print (min(valid_song['song_id']))

-1.0


In [ ]:
valid_freq = user_song_freq.loc[(user_song_freq['freq']<user_song_freq['freq'].quantile(0.999)) & (user_song_freq['song_id'] > 0)]
print (valid_freq.shape[0]/user_song_freq.shape[0])

In [ ]:
print (extreme_freq.shape)
print (len(np.unique(extreme_freq['uid']))/len(np.unique(valid_freq['uid'])))

In [44]:
print (valid_song.loc[valid_song['song_id'] == 160911])
temp_df = valid_song.loc[valid_song['song_id'] == 160911.0]
song_type = max((temp_df['song_type'].repalce(0.0, None)).mode, 0)
print (song_type)

          song_id  song_type song_name   singer  song_length
1        160911.0        NaN   蔡依林&周杰伦        0          NaN
80361    160911.0        0.0     布拉格广场  蔡依林&周杰伦        296.0
264168   160911.0        0.0     布拉格广场  蔡依林&周杰伦          0.0
343003   160911.0        0.0     布拉格广场  蔡依林&周杰伦        272.0
372496   160911.0        0.0     布拉格广场  蔡依林&周杰伦        181.0
386901   160911.0        0.0     布拉格广场  蔡依林&周杰伦        212.0
467896   160911.0        0.0     布拉格广场  蔡依林&周杰伦        284.0
570168   160911.0        0.0     布拉格广场  蔡依林&周杰伦        273.0
581175   160911.0        0.0     布拉格广场  蔡依林&周杰伦        294.0
655835   160911.0        0.0     布拉格广场  蔡依林&周杰伦        227.0
949938   160911.0        0.0     布拉格广场  蔡依林&周杰伦        242.0
1017397  160911.0        0.0     布拉格广场  蔡依林&周杰伦        241.0
1019248  160911.0        2.0     布拉格广场  蔡依林&周杰伦        292.0
1029274  160911.0        1.0     布拉格广场  蔡依林&周杰伦        296.0
1085010  160911.0        0.0     布拉格广场  蔡依林&周杰伦        232.0
1129586  160911.0       

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\ops.py:798: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(x, name)(y)


TypeError: invalid type comparison

In [72]:
temp_df = valid_song.loc[valid_song['song_id'] == 160911.0]
# temp = temp_df['song_type'].replace(0.0, None)
# temp.dropna().mode()
song_name = temp_df['song_name'].dropna().mode()
print (song_name)

0    布拉格广场
dtype: object


In [ ]:
print (len(np.unique(valid_freq['song_id'])))

### feature enigneering 
#### purpose useful features
- song freq for users  
- download for each users  
- various kinds of similarity matrix  

### modeling 
#### utility 
- for each user, each song: c0 (not played), c1 (play once only/dislike the song), c2 (play multiple times), c3 (love the song, download)
- collaborative filtering -> 
- relevance modeling (pair)